In [ ]:
import pandas as pd


targetGenes = ['TBC1D9', 'GATA3', 'SLC16A6', 'ESR1', 'INPP4B', 'SLC44A4', 'ANXA9', 'AGR2', 'MCCC2', 'TSPAN1', 'STBD1', 'MLPH', 'CACNA2D2',
'RARA', 'STARD3', 'PPP1R14C', 'SFRS13B', 'LDHB', 'MFGE8', 'PSAT1']

In [ ]:
df = pd.read_csv('../Data/clinical.csv')

In [ ]:
geneFiles = df[['file', 'tnbc', 'bcr_patient_uuid']]

In [ ]:
dfPatientGenes = pd.DataFrame()
total_files = len(geneFiles)
for idx, row in geneFiles.iterrows():
    file_name = row.iloc[0]
    file_path = f"../Data/{file_name}"
    
    print(f"File {idx+1}/{total_files} - {file_name}", end="\r")
    
    dfGenes = pd.read_csv(file_path, sep='\t', skiprows=[0])
    
    # Columns may have been dropped if first line in file were columns, load data again without dropping first line
    if "gene_name" not in dfGenes.columns:
        dfGenes = pd.read_csv(file_path, sep='\t')
    
    #dfGenes
    
    #Gather all targeted genes we want with the values we want
    dfTarget = dfGenes[dfGenes['gene_name'].isin(targetGenes)][['gene_name', 'stranded_first']]

    #Transform the dfTarget to a single row dataframe with all genes as columns
    dfNewGenes = dfTarget.set_index('gene_name').T.reset_index(drop=True)

    # Add tnbc status
    dfNewGenes["tnbc"] = df["tnbc"].loc[df['file'] == file_name].values

    #Add the patient uuid for reference and set it as the index
    patientID = df['bcr_patient_uuid'].loc[df['file'] == file_name]
    dfNewGenes['case_id'] = patientID.values
    
    #Concat the data to the final dataframe before saving it to a file
    dfPatientGenes = pd.concat([dfPatientGenes, dfNewGenes])
    
    
#For visualizing the result
# dfPatientGenes

In [ ]:
#Send data to a csv

dfPatientGenes.to_csv('../Data/patient_genes.csv', index=False)